In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import  OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import pylab

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
original_data = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

In [ ]:
def processing_data(data,examples):
    two = ['sex','cp','fbs','restecg','exng','slp','caa','thall']
    col1 = np.zeros(shape=(examples,1))
    for i in two:
        encoder = preprocessing.OneHotEncoder()
        hh = encoder.fit_transform(data[i].values.reshape(-1,1)).toarray()
        col1 = np.column_stack((col1,hh))
    col1 = np.delete(col1,0,axis=1)
    
    rest = ['age','trtbps','chol','thalachh','oldpeak','output']
    col = np.zeros(shape=(303,1))
    
    col = np.zeros(shape=(examples,1))
    for i in rest:
        col = np.column_stack((col,data[i].values.reshape(-1,1)))    
    ss = preprocessing.StandardScaler()
    col = preprocessing.scale(col)
    col = ss.fit_transform(col)    
    col = np.delete(col,0,axis=1)
    col_new = np.column_stack((col1,col))   
    return col_new

In [ ]:
def estimater_model(estimater,X_train,y_train,X_test,y_test,table,esti):
    result =estimater.fit(X_train,y_train) 
    score = estimater.score(X_test,y_test)
    cross = cross_val_score(estimater, X_train, y_train, cv=20)
    print('%s:'% (esti))
    print(cross.mean())
    print(estimater.get_params)
    print("\n")

    table.loc[esti,'accuracy_score'] = score
    
#     name = './Documents/Train_Model_result/'+esti +'_train_model.pkl'
#     joblib.dump(result, name)
    return table,cross.max()

In [ ]:
def predict_result(estimater,X_test,y_test,ax):
    y_predict = estimater.predict(X_test)
    pylab.rcParams['figure.figsize'] = (100.0, 100.0)
#     ax.title("predict and real")
    ax.plot(range(len(X_test)),y_predict, color='skyblue', label='y_predict')
    ax.plot(range(len(X_test)),y_test, color='red', label='y_test')
    ax.legend()

In [ ]:
data = processing_data(original_data,303)
target = data[:,-1].astype('int')
data = np.delete(data,-1,axis=1)
X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.3,random_state=42)

In [ ]:
estimaters = ['SVC','DecisiontreeClassifier','RandomForestRegressor',
             'DecisionTreeRegressor','LinearRegression','Ridge',
             'Lasso','BayesianRidge','LogisticRegression',
             'GaussianNB','SGDClassifier']
params = [{'C':10,'kernel':'poly','gamma':'auto'},
       {'max_depth':5,'min_samples_split':3,'min_samples_leaf':2},
       {'n_estimators':200,'criterion':'mse','min_samples_split':4},
          {'criterion':'mse','max_features':'log2','max_depth':50}, #,'n_estimators':100
          {},
          {},
          {},
          {},
          {},
          {},
          {}]

In [ ]:
table = pd.DataFrame(columns=['accuracy_score'],index=estimaters)
for esti,pa,i in zip(estimaters,params,range(1,11)):    
    if esti =='SVC':
        estimator = SVC(**pa)
    elif esti == 'DecisiontreeClassifier':
        estimator = tree.DecisionTreeClassifier(**pa)
    elif esti == 'DecisionTreeRegressor':
        estimator = tree.DecisionTreeRegressor(**pa)
    elif esti == 'RandomForestRegressor':
        estimator = RandomForestRegressor(**pa)
    elif esti == 'LinearRegression':
        estimator = linear_model.LinearRegression(**pa)
    elif esti == 'Ridge':
        estimator = linear_model.Ridge(**pa)
    elif esti == 'Lasso':
        estimator = linear_model.Lasso(**pa)
    elif esti == 'BayesianRidge':
        estimator = linear_model.BayesianRidge(**pa)
    elif esti == 'LogisticRegression':
        estimator = linear_model.LogisticRegression(**pa)
    elif esti == 'GaussianNB':
        estimator = GaussianNB(**pa)
    elif esti == 'SGDClassifier':
        estimator = linear_model.SGDClassifier(**pa)
    if estimator != 'NuLL':
        table_new,cross = estimater_model(estimator,X_train,y_train,X_test,y_test,table,esti)
        ax = plt.subplot(3,4,i)
        predict_result(estimator,X_test,y_test,ax)
print("\n")
print(table_new)